# VETRA – Sistema Multiagente com LangGraph + Google Gemini

Este notebook implementa um MVP funcional do VETRA, um sistema multiagente para avaliação de risco e viabilidade no contexto cripto.

In [1]:
import os
import re
import json
import logging
import time
import random
from typing import Dict, List, Optional, TypedDict, Any
from dataclasses import dataclass
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END

from dataclasses import dataclass

In [2]:
import os
import re
import json
import logging
import time
import random
from typing import Dict, List, Optional, TypedDict, Any
from dataclasses import dataclass
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END

from dataclasses import dataclass

In [ ]:
# Configuração básica de logs
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

load_dotenv()

logging.info("Inicializando o modelo Google Gemini...")
llm = None

try:
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        logging.warning("A variável GOOGLE_API_KEY não foi encontrada no .env. O LLM não estará disponível.")
    else:
        llm = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash",     # Você pode trocar para "gemini-1.5-flash" (mais rápido, menor custo)
            temperature=0.2,
            google_api_key=GOOGLE_API_KEY,
            convert_system_message_to_human=True
        )
        logging.info("Modelo Google Gemini inicializado com sucesso.")
except Exception as e:
    logging.error(f"Erro ao inicializar o Gemini: {e}")


In [77]:
class VetraState(TypedDict, total=False):
    query: str                      # Consulta original do usuário
    route: List[str]                # Sequência de agentes a executar
    scores: Dict[str, float]        # Pontuações (0..1) de cada agente
    rationales: Dict[str, str]      # Explicações curtAS de cada agente
    evidence: List[str]             # Evidências agregadas pelos agentes
    final_report: str               # Relatório final consolidado
    final_json: Dict[str, Any]      # Saída JSON final estruturada


# tools

In [ ]:

#todo: precisa substituir por integrações reais

@dataclass
class WebResult:
    title: str
    url: str
    snippet: str

def web_scrape(query: str) -> List[WebResult]:
    """substituir por coisas reais"""
    return [
        WebResult(title="Token X – Site Oficial", url="https://example.com", snippet="Contrato auditado e liquidez travada?"),
        WebResult(title="Discussão em Fórum", url="https://forum.example/x", snippet="Relatos de possíveis problemas antigos.")
    ]

def fetch_private_data(identifier: str) -> Dict:
    """apenas simulcao, precisa substituir por coisas reais"""
    return {
        "identifier": identifier,
        "holders_top10": 0.76,
        "lp_locked_days": 2,
        "tx_velocity": 1.8,
        "age_days": 11,
    }

def math_estimator(features: Dict) -> float:
    """calculo heurístico simples de risco (0..1), precisa substituir por algo mais sofisticado"""
    risk = 0.0
    risk += min(1.0, features.get("holders_top10", 0) * 0.8)
    risk += 0.2 if features.get("lp_locked_days", 0) < 7 else 0.0
    risk += 0.1 if features.get("age_days", 0) < 14 else 0.0
    return max(0.0, min(1.0, risk))


# prompts

In [ ]:

SYSTEM_SUP = SystemMessage(content=(
    'Você é o Supervisor do VETRA. Com base na consulta do usuário, consulte linearmente os agentes que vão ser utilizados.'
    'Agentes disponíveis: phishing, transaction, rugpull.'
    'Responda APENAS com um JSON válido exatamente neste formato: {"route":["phishing","rugpull","transaction"]}.'
    "Inclua somente a chave 'route' com os nomes válidos em minúsculas." \
    "Cada agente irá retornar um SCORE(0..1) e uma explicação curta, onde 0 é uma análise segura para aquele tipo de golpe e 1 é o maior risco possível." \
    "Seu trabalho é consolidar os resultados em um relatório final e uma saída JSON estruturada." \
    "No relatório final, inclua:\n" \
    "1) Uma visão geral do risco total (0..1) com três casas decimais realizando a média dos scores dos agentes.\n" \
    "2) Um resumo breve (máx. 6 frases) destacando os principais fatores de risco identificados pelos agentes.\n" \
    "Na saída JSON, inclua:\n" \
    "1) 'final_score': risco total (0..1) com três casas decimais. este risco é a média da pontuação de cada agente\n" \
    "2) 'agent_scores': dicionário com scores individuais de cada agente.\n" \
    "3) 'rationales': dicionário com explicações curtas de cada agente.\n" \
    "4) 'explanation': relatório final consolidado, explicando se aquela transação é segura baseada na pontuação total." \
    "Exemplo de ordens de escolhas de agentes:\n" \
    '{"route":["phishing", "rugpull", "transaction"]}\n' \
    "Exemplo de saída JSON final:\n" \
    '{\n' \
    '  "final_score": 0.725,\n' \
    '  "agent_scores": {\n' \
    '    "phishing": 0.900,\n' \
    '    "rugpull": 0.600,\n' \
    '    "transaction": 0.675\n' \
    '  },\n' \
    '  "rationales": {\n' \
    '    "phishing": "A mensagem contém um link encurtado e solicita informações pessoais, indicando um alto risco de phishing.",\n' \
    '    "rugpull": "O token apresenta alta concentração de supply em poucas carteiras e a liquidez não está bloqueada, sugerindo um risco moderado de rugpull.", \n' \
    '    "transaction": "A transação está associada a carteiras sancionadas e apresenta padrões de anomalia, resultando em um risco significativo. "\n' \
    '  },\n' \
    '  "explanation": "O risco total da transação é 0.725, indicando um nível considerável de risco. O alto risco de phishing devido a links suspeitos e solicitações de informações pessoais é preocupante. '
    'Além disso, o token avaliado apresenta sinais de rugpull com alta concentração de supply e falta de liquidez bloqueada. A associação da transação com carteiras sancionadas também contribui para o risco geral. Recomenda-se cautela ao interagir com esta transação."\n' \
))

SYSTEM_AGENT = SystemMessage(content=(
    "Você é um analista de risco cripto do VETRA. Sempre responda com:\n"
    "1) SCORE(0..1) na primeira linha (com três casas decimais)\n"
    "2) Em seguida, uma explicação curta (no máximo 4 frases)."
))

SYSTEM_PHISHING = SystemMessage(content=(
    "Você é um analista de risco especializado em PHISHING para o VETRA.\n"
    "OBJETIVO: estimar o risco de phishing em uma mensagem ou transação.\n"
    "USE SÓ: evidências fornecidas (snippets de busca, reputação, sinais de engano) — não invente.\n"
    "CRITÉRIOS/RED FLAGS: domínios parecidos (typosquatting), URLs encurtadas, HTTPS falso/inconsistente, "
    "marca mal utilizada, urgência suspeita, solicitação de seed/privadas, formulários pedindo chaves, "
    "DNS jovem, ausência de perfis oficiais, links levando a carteiras desconhecidas, pedidos de informações pessoais.\n"
    "OUTPUT OBRIGATÓRIO (JSON válido):\n"
    "{\n"
    "  \"agent_analysis\": {\n"
    "    \"token_agent\": {\n"
    "      \"score\": <float 0..1>,\n"
    "      \"findings\": [\"bullet points curtos e objetivos\"],\n"
    "      \"severity\": \"low\" | \"medium\" | \"high\"\n"
    "    }\n"
    "  },\n"
    "  \"factor\": \"transaction\",\n"
    "  \"severity\": \"low\" | \"medium\" | \"high\",\n"
    "  \"description\": \"Resumo objetivo de 1–3 frases para usuário final.\"\n"
    "}\n"
"FORMATAÇÃO: Retorne APENAS o JSON. Sem texto adicional."
"EXEMPLO DE OUTPUT OBRIGATÓRIO:\n"
"agent_analysis" {
    "token_agent": {
        "score": 0.125,
        "findings": [
            "A mensagem não contém links encurtados nem solicita informações pessoais sensíveis. O domínio parece legítimo e utiliza HTTPS corretamente. Não há sinais de urgência suspeita ou formulários pedindo chaves privadas."
        ],
        "severity": "low"
    },  
    "factor": "phishing",
    "severity": "low",
    "description": "A análise indica um risco baixo de phishing. A mensagem parece confiável com base nas evidências fornecidas."        
}
""
"EXEMPLO DE OUTPUT OBRIGATÓRIO:\n"
"agent_analysis" {
    "token_agent": {
        "score": 0.900,
        "findings": [
            "A mensagem contém um link encurtado (bit.ly/xyz123) e solicita informações pessoais, o que são sinais claros de phishing. O domínio utilizado é suspeito e não corresponde à marca oficial. Além disso, há uma sensação de urgência na mensagem, pressionando o usuário a agir rapidamente."
        ],
        "severity": "high"
    },  
    "factor": "phishing",
    "severity": "high",
    "description": "A análise indica um risco alto de phishing devido à presença de links encurtados, solicitações de informações pessoais e sinais de urgência na mensagem."
     }"
))

SYSTEM_TRANSACTION = SystemMessage(content=(
    "Você é um analista de risco on-chain do VETRA focado em TRANSAÇÕES E CONTRATOS ENVOLVIDOS NA TX.\n"
    "OBJETIVO: estimar o risco da TRANSAÇÃO atual e dos CONTRATOS tocados por ela.\n"
    "ESCOPO (inclui, mas não se limita a):\n"
    "- Tipo de tx (swap, transfer, mint/burn, stake/unstake, approve/permit/delegate, create account, program invoke).\n"
    "- Funções chamadas e permissões concedidas (approve ilimitado, permit, delegate, setAuthority, pause/blacklist, upgrade/proxy, revoke ausente).\n"
    "- Contratos/programas tocados: verificado? proxy/upgradeable? pausable/blacklist? taxa/fee-on-transfer? transfer-hook? freeze/mint authority?\n"
    "- Contrapartes e rotas: CEX/mixer/sanção/endereços marcados; DEX/routers suspeitos; MEV/sandwich; flashloans; slippage incomum.\n"
    "- Padrões anômalos: bursts, wallet jovem, repetição de pequenos montantes, fan-in/out concentrado.\n"
    "DISTINÇÃO: NÃO avalie risco estrutural do TOKEN (liquidez bloqueada, distribuição de holders etc.) — isso é do agente RUGPULL. "
    "Se surgir, cite como 'out-of-scope' e não pese no score.\n"
    "NÃO INVENTE: use SOMENTE os dados fornecidos no prompt.\n"
    "SE O COMPORTAMENTO FOR NORMAL, atribua score baixo.\n"
    "OUTPUT OBRIGATÓRIO (JSON válido):\n"
    "{\n"
    "  \"agent_analysis\": {\n"
    "    \"token_agent\": {\n"
    "      \"score\": <float 0..1>,\n"
    "      \"findings\": [\"bullet points curtos e objetivos\"],\n"
    "      \"severity\": \"low\" | \"medium\" | \"high\"\n"
    "    }\n"
    "  },\n"
    "  \"factor\": \"transaction\",\n"
    "  \"severity\": \"low\" | \"medium\" | \"high\",\n"
    "  \"description\": \"Resumo objetivo de 1–3 frases para usuário final.\"\n"
    "}\n"
"FORMATAÇÃO: Retorne APENAS o JSON. Sem texto adicional."
"EXEMPLO DE OUTPUT OBRIGATÓRIO:\n"
"agent_analysis" {
    "token_agent": {
        "score": 0.750,
        "findings": [
            "A transação envolve um swap em um DEX não verificado, o que aumenta o risco.\n",
            "O contrato chamado possui funções de upgrade e pausabilidade, o que pode ser explorado por agentes maliciosos.\n",
            "A carteira de origem é jovem (15 dias) e apresenta um padrão de transações anômalas, sugerindo possível atividade suspeita."
        ],
        "severity": "high"
    },  
    "factor": "transaction",
    "severity": "high",
    "description": "A análise indica um risco alto para esta transação devido ao uso de um DEX não verificado, funções de contrato potencialmente perigosas e padrões anômalos na carteira de origem."
     },"
"EXEMPLO DE OUTPUT OBRIGATÓRIO:\n"
"agent_analysis" {
    "token_agent": {
        "score": 0.100,
        "findings": [
            "A transação é uma simples transferência entre carteiras verificadas, sem envolvimento de DEXs ou contratos complexos.\n",
            "Nenhuma função de alto risco foi chamada, e a carteira de origem tem um histórico limpo e estabelecido.\n",
            "Não foram detectados padrões anômalos nas transações recentes da carteira."
        ],
        "severity": "low"
    },  
    "factor": "transaction",
    "severity": "low",
    "description": "A análise indica um risco baixo para esta transação, que é uma transferência simples entre carteiras verificadas sem sinais de atividade suspeita."
    }   
))

SYSTEM_RUGPULL = SystemMessage(content=(
    "Você é um analista de tokens para o VETRA focado em RUGPULL.\n"
    "OBJETIVO: estimar risco de rugpull de um token.\n"
    "USE SÓ: evidências fornecidas (features on-chain, dados de contratos, discussões públicas) — não invente.\n"
    "CRITÉRIOS/RED FLAGS: liquidez bloqueada/desbloqueio próximo, alta taxa de imposto, mint authority ativa, "
    "ownership não renunciada, concentração de supply em poucas carteiras, liquidez baixa, histórico de pulls "
    "do deployer, alterações recentes em permissões, trading desabilitado, honeypot signals.\n"
    "Você receberá features e uma heurística preliminar (0..1). Considere ambas.\n"
    "OUTPUT OBRIGATÓRIO (JSON válido):\n"
    "{\n"
    "  \"agent_analysis\": {\n"
    "    \"token_agent\": {\n"
    "      \"score\": <float 0..1>,\n"
    "      \"findings\": [\"bullet points curtos e objetivos\"],\n"
    "      \"severity\": \"low\" | \"medium\" | \"high\"\n"
    "    }\n"
    "  },\n"
    "  \"factor\": \"transaction\",\n"
    "  \"severity\": \"low\" | \"medium\" | \"high\",\n"
    "  \"description\": \"Resumo objetivo de 1–3 frases para usuário final.\"\n"
    "}\n"
"FORMATAÇÃO: Retorne APENAS o JSON. Sem texto adicional."
"EXEMPLO DE OUTPUT OBRIGATÓRIO:\n"
"agent_analysis" {
    "token_agent": {
        "score": 0.850,
        "findings": [
            "A análise revela várias red flags significativas. A concentração de holders no top 10 é alta (76%), indicando risco de manipulação. A liquidez está bloqueada por apenas 2 dias, com desbloqueio iminente, aumentando o risco de rugpull. A idade do token é baixa (11 dias), o que dificulta a avaliação da sua confiabilidade a longo prazo."
        ],
        "severity": "high"
    },  
    "factor": "rugpull",
    "severity": "high",
    "description": "A análise indica um risco alto de rugpull porque a concentração de supply no top 10 é elevada (76%), a liquidez está prestes a ser desbloqueada (em 2 dias), e o token é relativamente novo (11 dias), o que aumenta a incerteza sobre sua estabilidade futura."
     },"
"EXEMPLO DE OUTPUT OBRIGATÓRIO:\n"
"agent_analysis" {
    "token_agent": {
        "score": 0.025,
        "findings": [
            "A análise indica um risco muito baixo de rugpull. A liquidez está totalmente bloqueada por 180 dias, o que é um forte indicador de segurança. A concentração de supply no top 10 é baixa (15%), sugerindo uma distribuição saudável entre os holders. Além disso, o token tem uma idade considerável (250 dias), o que contribui para a confiança na sua estabilidade a longo prazo."
        ],
        "severity": "low"
    },
    "factor": "rugpull",
    "severity": "low",
    "description": "A análise indica um risco muito baixo de rugpull devido à liquidez totalmente bloqueada, baixa concentração de supply no top 10 e idade considerável do token."
    }
))




# agents

In [ ]:
def supervisor_node(state: VetraState) -> VetraState:
    """Decide a lista de agentes a executar com base na query do usuário"""
    user_q = state["query"].strip()
    msg = [SYSTEM_SUP, HumanMessage(content=user_q)]
    raw = llm.invoke(msg).content if llm else "phishing,transaction,rugpull"
    csv = raw.strip().lower()

    valid = []
    for part in re.split(r"[\,\n;\s]+", csv):
        if part in {"phishing", "transaction", "rugpull"} and part not in valid:
            valid.append(part)
    if not valid:
        valid = ["phishing"]

    return {**state, "route": valid, "scores": {}, "rationales": {}, "evidence": []}


def phishing_agent_node(state: VetraState) -> VetraState:
    query = state["query"]
    results = web_scrape(query)
    snippets = "\n".join([f"- {r.title} | {r.url} | {r.snippet}" for r in results])

    prompt = f"Analise possível phishing relacionado a: {query}.\nEvidências:\n{snippets}\nAvalie o risco."
    out = llm.invoke([SYSTEM_PHISHING, HumanMessage(content=prompt)]).content.strip() if llm else "0.2\nSite parece legítimo e usa SSL válido."
    score = _extract_score(out)
    rationale = out.split("\n", 1)[1] if "\n" in out else ""

    state["scores"]["phishing"] = score
    state["rationales"]["phishing"] = rationale
    state["evidence"].append(snippets)
    
    route = state.get("route", [])
    if "phishing" in route:
        route.remove("phishing")
        state["route"] = route
    
    return state


def transaction_agent_node(state: VetraState) -> VetraState:
    identifier = _first_wallet_or_tx(state["query"]) or "unknown"
    feats = fetch_private_data(identifier)

    prompt = f"Analise comportamento on-chain. Dados: {feats}\nForneça SCORE de risco e uma justificativa curta."
    out = llm.invoke([SYSTEM_TRANSACTION, HumanMessage(content=prompt)]).content.strip() if llm else "0.7\nTransações concentradas em poucas carteiras."
    score = _extract_score(out)
    rationale = out.split("\n", 1)[1] if "\n" in out else ""

    state["scores"]["transaction"] = score
    state["rationales"]["transaction"] = rationale
    state["evidence"].append(str(feats))
    
    route = state.get("route", [])
    if "transaction" in route:
        route.remove("transaction")
        state["route"] = route
    
    return state


def rugpull_agent_node(state: VetraState) -> VetraState:
    identifier = _first_token(state["query"]) or "tokenX"
    feats = fetch_private_data(identifier)
    heuristic = math_estimator(feats)

    prompt = f"Analise risco de rugpull. Features: {feats}\nHeurística preliminar: {heuristic:.3f}\nForneça SCORE final e justificativa curta."
    out = llm.invoke([SYSTEM_RUGPULL, HumanMessage(content=prompt)]).content.strip() if llm else "0.9\nLiquidez destrava em breve; risco elevado."
    score = _extract_score(out)
    rationale = out.split("\n", 1)[1] if "\n" in out else ""

    # Combinação simples entre o score do modelo e a heurística para estabilidade
    score = float(min(1.0, max(0.0, 0.7 * score + 0.3 * heuristic)))

    state["scores"]["rugpull"] = score
    state["rationales"]["rugpull"] = rationale
    state["evidence"].append(str(feats))
    
    route = state.get("route", [])
    if "rugpull" in route:
        route.remove("rugpull")
        state["route"] = route
    
    return state


In [ ]:

def _extract_score(text: str) -> float:
    """Extrai o valor SCORE (0..1) da primeira linha de texto do agente."""
    first = text.split("\n", 1)[0]
    m = re.search(r"([01](?:\.\d+)?)", first)
    try:
        val = float(m.group(1)) if m else 0.5
    except Exception:
        val = 0.5
    return max(0.0, min(1.0, val))


def _first_wallet_or_tx(q: str) -> Optional[str]:
    m = re.search(r"0x[a-fA-F0-9]{6,}", q)
    return m.group(0) if m else None


def _first_token(q: str) -> Optional[str]:
    m = re.search(r"\b[A-Z]{2,10}\b", q)
    return m.group(0) if m else None


In [ ]:
def trust_index_node(state: VetraState) -> VetraState:
    """Agrega os resultados dos agentes e produz um relatório final e valores em JSON."""
    scores = state.get("scores", {})
    weights = {"phishing": 0.35, "transaction": 0.30, "rugpull": 0.35}

    num = sum(weights.get(k, 0.0) * v for k, v in scores.items())
    den = sum(weights.values())
    trust_index = num / den if den else 0.0

    bullets = [f"- {k.title()}: {scores[k]:.3f} — {state['rationales'].get(k, '')}" for k in scores]
    report = f"VETRA Trust Index: {trust_index:.3f} (0 = seguro, 1 = risco alto)\n\n" + "\n".join(bullets)
    state["final_report"] = report

    # JSON estruturado (guardado no estado para exibição posterior)
    state["final_json"] = {
        "trust_index": round(trust_index, 3),
        "scores": {k: round(v, 3) for k, v in scores.items()},
        "rationales": state.get("rationales", {}),
        "evidence": state.get("evidence", []),
        "final_report": report
    }
    return state


In [ ]:
def build_graph():
    graph = StateGraph(VetraState)
    graph.add_node("supervisor", supervisor_node)
    graph.add_node("phishing", phishing_agent_node)
    graph.add_node("transaction", transaction_agent_node)
    graph.add_node("rugpull", rugpull_agent_node)
    graph.add_node("trust_index", trust_index_node)

    graph.set_entry_point("supervisor")

    def choose_next(state: VetraState):
        route = state.get("route", [])
        if not route:
            return "trust_index"
        nxt = route.pop(0)
        state["route"] = route
        return nxt

    def route_after_agent(state: VetraState):
        route = state.get("route", [])
        if not route:
            return "trust_index"
        return "supervisor"

    graph.add_conditional_edges("supervisor", choose_next, {
        "phishing": "phishing",
        "transaction": "transaction", 
        "rugpull": "rugpull",
        "trust_index": "trust_index",
    })

    for node in ["phishing", "transaction", "rugpull"]:
        graph.add_conditional_edges(node, route_after_agent, {
            "supervisor": "supervisor",
            "trust_index": "trust_index"
        })

    graph.add_edge("trust_index", END)
    return graph.compile()


In [ ]:
app = build_graph()

# Exemplo de consulta 
consulta = "Analise o token ABC 0x1234567890 e verifique risco de rugpull e phishing."

resultado = app.invoke({"query": consulta})

# Exibir relatório legível
print("--------- RELATÓRIO FINAL ---------")
print(resultado.get("final_report", "(nenhum relatório gerado)"))
print("-----------------------------------\n")

# Exibir JSON estruturado
final_json = resultado.get("final_json", {})
print("--------- SAÍDA JSON ---------")
print(json.dumps(final_json, ensure_ascii=False, indent=2))
print("--------------------------------")


--------- RELATÓRIO FINAL ---------
VETRA Trust Index: 0.350 (0 = seguro, 1 = risco alto)

- Phishing: 1.000 — 
2) O token ABC (0x1234567890) apresenta risco moderado. Embora o site oficial (example.com) indique auditoria e liquidez travada, a discussão em fórum (forum.example/x) levanta preocupações com relatos de problemas antigos. É crucial investigar a natureza desses problemas e a reputação da equipe por trás do token para mitigar o risco de rugpull ou phishing. A auditoria e o bloqueio de liquidez são positivos, mas não eliminam totalmente o risco.
-----------------------------------

--------- SAÍDA JSON ---------
{
  "trust_index": 0.35,
  "scores": {
    "phishing": 1.0
  },
  "rationales": {
    "phishing": "\n2) O token ABC (0x1234567890) apresenta risco moderado. Embora o site oficial (example.com) indique auditoria e liquidez travada, a discussão em fórum (forum.example/x) levanta preocupações com relatos de problemas antigos. É crucial investigar a natureza desses problem

In [ ]:
from typing import Dict

DEFAULT_WEIGHTS = {
    "phishing": 0.95,     # phishing costuma ser “fatal” p/ segurança do usuário
    "transaction": 0.75,  # risco comportamental on-chain
    "rugpull": 0.85,      # tokenomics/liquidez perigosos
}

OVERRIDES = [
    ("phishing",   0.90, 0.98),
    ("rugpull",    0.90, 0.95),
    ("transaction",0.90, 0.95),
]

def aggregate_final_score(
    scores: Dict[str, float],
    weights: Dict[str, float] = None,
    overrides = None
) -> float:
    """
    scores: {"phishing": s_p, "transaction": s_t, "rugpull": s_r} com valores em [0,1].
    Retorna score final em [0,1], monotônico e “puxado para 1” quando há alto risco em um agente.
    """
    weights = weights or DEFAULT_WEIGHTS
    overrides = overrides or OVERRIDES

    # 1) Noisy-OR ponderado
    prod = 1.0
    for k, s in scores.items():
        w = float(weights.get(k, 1.0))
        s_clamped = min(1.0, max(0.0, s))
        term = max(0.0, 1.0 - w * s_clamped)
        prod *= term
    final_score = 1.0 - prod

    # 2) Overrides (short-circuit) para casos críticos
    for k, threshold, floor_value in overrides:
        if scores.get(k, 0.0) >= threshold:
            final_score = max(final_score, floor_value)

    # 3) Clamp final por segurança
    return float(min(1.0, max(0.0, final_score)))


In [ ]:
def test_valid_scores():
    results = []

    # Caso A: todos os scores em zero → risco mínimo
    scoresA = {"phishing": 0.0, "transaction": 0.0, "rugpull": 0.0}
    results.append(aggregate_final_score(scoresA))

    # Caso B: todos os scores em um → risco máximo
    scoresB = {"phishing": 1.0, "transaction": 1.0, "rugpull": 1.0}
    results.append(aggregate_final_score(scoresB))

    # Caso C: phishing médio, outros baixos
    scoresC = {"phishing": 0.5, "transaction": 0.2, "rugpull": 0.1}
    results.append(aggregate_final_score(scoresC))

    # Caso D: rugpull alto, outros médios
    scoresD = {"phishing": 0.4, "transaction": 0.6, "rugpull": 0.91}
    results.append(aggregate_final_score(scoresD))

    # Caso E: transaction no limite do override
    scoresE = {"phishing": 0.3, "transaction": 0.90, "rugpull": 0.2}
    results.append(aggregate_final_score(scoresE))

    # Caso F: phishing no limite inferior do override
    scoresF = {"phishing": 0.90, "transaction": 0.4, "rugpull": 0.4}
    results.append(aggregate_final_score(scoresF))

    # Caso G: valores intermediários sem ativar override
    scoresG = {"phishing": 0.6, "transaction": 0.6, "rugpull": 0.6}
    results.append(aggregate_final_score(scoresG))

    # Caso H: Risco leve em todos os agentes
    scoresH = {"phishing": 0.2, "transaction": 0.2, "rugpull": 0.2}
    results.append(aggregate_final_score(scoresH))

    # Caso I: Risco leve em dois agentes, moderado em phishing
    scoresH = {"phishing": 0.4, "transaction": 0.2, "rugpull": 0.2}
    results.append(aggregate_final_score(scoresH))

    # Imprimir resultados
    for i, r in enumerate(results, 1):
        print(f"Caso {chr(64+i)}: Score final = {r:.4f}")
test_valid_scores()

Caso A: Score final = 0.0000
Caso B: Score final = 0.9981
Caso C: Score final = 0.5917
Caso D: Score final = 0.9500
Caso E: Score final = 0.9500
Caso F: Score final = 0.9800
Caso G: Score final = 0.8841
Caso H: Score final = 0.4285
Caso I: Score final = 0.5626
